# Домашнее задание №1: линейная регрессия и векторное дифференцирование.


In [1]:
import numpy as np

## Многомерная линейная регрессия из sklearn

Применим многомерную регрессию из sklearn для стандартного датасета

In [2]:
from sklearn.datasets import make_regression

X, y = make_regression(n_samples = 10000)
print(X.shape, y.shape)

(10000, 100) (10000,)


У нас 10000 объектов и 100 признаков. Для начала решим задачу аналитически "из коробки".

In [3]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

reg = LinearRegression().fit(X, y)
print(mean_squared_error(y, reg.predict(X)))

1.4153191393409055e-25


Теперь попробуем обучить линейную регрессию методом градиентного спуска "из коробки"

In [4]:
from sklearn.linear_model import SGDRegressor
reg = SGDRegressor(alpha=0.00000001).fit(X, y)
print(mean_squared_error(y, reg.predict(X)))
reg.coef_

4.246479180400162e-12


array([-8.38654327e-08, -5.51883044e-09, -2.13333867e-08,  1.73398485e-08,
        1.08029089e-08,  2.21901699e-08, -3.16431505e-08, -1.45648663e-08,
       -1.09481239e-08, -1.84244413e-08, -2.59007135e-08,  7.68953347e+00,
        3.08940798e-08, -2.03821823e-08,  3.51693609e-08,  1.74448130e-08,
       -3.65375575e-08,  2.43080626e-08, -4.43754989e-09,  5.28439206e-08,
       -5.95282914e-09, -3.84824518e-08, -2.03121756e-08,  3.76657526e-08,
       -1.62217886e-08,  1.76704332e-08,  1.22977865e-08, -1.34962950e-08,
        8.83785713e+01,  7.87209139e-09,  3.90931070e-08,  3.73041780e-08,
        1.85952133e+01, -4.78183369e-08,  7.35388056e-09,  2.15118122e-08,
       -1.42266033e-08,  4.77847181e-08,  7.27979395e+01,  2.25597834e-08,
       -5.27757582e-09, -4.20839142e-08,  6.46997312e-09, -5.06536030e-08,
       -4.54787493e-08, -2.68223563e-08,  2.89814406e-08,  5.92976645e-08,
        1.59055122e-08,  7.02494999e+01,  2.93440801e-08,  3.16567019e-08,
        6.96770111e-08, -

***Задание 1 (0.5 балла).*** Объясните, чем вызвано различие двух полученных значений метрики?

***Задание 2 (0.5 балла).*** Подберите гиперпараметры в методе градиентного спуска так, чтобы значение MSE было близко к значению MSE, полученному при обучении LinearRegression.

***Ответ 1:***

Для ```SGDRegressor``` выбрано слишком большое значение параметра регуляризации ```alpha```, которое может привести к медленной сходимости модели или даже застреванию в локальном минимуме. Кроме того, в методе градиентного спуска (```SGDRegressor```) веса инициализируются случайным образом, это может привести к тому, что модель начинает обучение не с оптимальных весов. В аналитическом решении (```LinearRegression```) веса инициализируются оптимально сразу, используя формулу нормального уравнения.

***Ответ 2:***

In [5]:
# Подбираем параметр alpha для SGDRegressor
reg = LinearRegression().fit(X, y)
mse_reg = mean_squared_error(y, reg.predict(X))

alpha = 0.1
sgd_reg = SGDRegressor(alpha=alpha).fit(X, y)
mse_sgd_reg = mean_squared_error(y, sgd_reg.predict(X))

while (mse_sgd_reg - mse_reg) > 1e-24:
    alpha /= 10
    sgd_reg = SGDRegressor(alpha=alpha).fit(X, y)
    mse_sgd_reg = mean_squared_error(y, sgd_reg.predict(X))
    print(f'α = {alpha}; MSE (SGDRegressor) = {mse_sgd_reg}\n')

α = 0.01; MSE (SGDRegressor) = 4.529973255985326

α = 0.001; MSE (SGDRegressor) = 0.044859996691931886

α = 0.0001; MSE (SGDRegressor) = 0.00043487612105318316

α = 1e-05; MSE (SGDRegressor) = 4.492913102550342e-06

α = 1.0000000000000002e-06; MSE (SGDRegressor) = 4.339679313498995e-08

α = 1.0000000000000002e-07; MSE (SGDRegressor) = 4.1074745160134284e-10

α = 1.0000000000000002e-08; MSE (SGDRegressor) = 4.24515358240008e-12

α = 1.0000000000000003e-09; MSE (SGDRegressor) = 4.2153691302830133e-14

α = 1.0000000000000003e-10; MSE (SGDRegressor) = 4.418123569810853e-16

α = 1.0000000000000003e-11; MSE (SGDRegressor) = 4.513738059356373e-18

α = 1.0000000000000002e-12; MSE (SGDRegressor) = 5.172318323009637e-20

α = 1.0000000000000002e-13; MSE (SGDRegressor) = 1.3823395763244954e-21

α = 1.0000000000000002e-14; MSE (SGDRegressor) = 7.5244534761252305e-25



С уменьшением ```alpha```, уменьшается ```MSE```

In [6]:
# Обучаем при alpha=1e-14
reg = SGDRegressor(alpha=1e-14).fit(X, y)
print(mean_squared_error(y, reg.predict(X)))

8.377705537895378e-25


In [7]:
# Обучаем при alpha=0
reg = SGDRegressor(alpha=0).fit(X, y)
print(mean_squared_error(y, reg.predict(X)))

8.610094468791942e-25


## Ваша многомерная линейная регрессия

***Задание 3 (5 баллов)***. Напишите собственную многомерную линейную регрессию, оптимизирующую MSE методом *градиентного спуска*. Для этого используйте шаблонный класс.

Критерий останова: либо норма разности весов на текущей и предыдущей итерациях меньше определенного значения (первый и третий варианты), либо модуль разности функционалов качества (MSE) на текущей и предыдущей итерациях меньше определенного значения (второй и четвертый варианты). Также предлагается завершать обучение в любом случае, если было произведено слишком много итераций.

***Задание 4 (2 балла)***. Добавьте l1 регуляризацию.

In [8]:
import numpy as np

class LinearRegression:
    def __init__(self, alpha=0.0001, l_ratio=0.001, tol=0.001, max_iter=1000):
        """
        Для начала необходимо инициализировать параметры
        alpha - это learning rate или шаг обучения
        l_ratio - коэффициент L1 регуляризации
        tol - значение для критерия останова
        max_iter - максимальное количество итераций обучения
        """
        self.alpha = alpha
        self.l_ratio = l_ratio
        self.tol = tol
        self.max_iter = max_iter

    def fit(self, X, y):
        """
        Метод для обучения линейной регрессии
        X - матрица признаков
        y - вектор правильных ответов
        """
        # Добавляем столбец единиц для учета свободного члена
        X = np.hstack((np.ones((X.shape[0], 1)), X))
        
        # Инициализация случайных весов
        self.weights = np.random.randn(X.shape[1])

        for _ in range(self.max_iter):
            y_pred = np.dot(X, self.weights)
            gradient = np.dot(X.T, (y_pred - y)) / X.shape[0]

            # L1-регуляризация для весов, исключая свободный член
            gradient[1:] += self.l_ratio * np.sign(self.weights[1:])

            prev_weights = self.weights.copy()
            self.weights -= self.alpha * gradient

            if np.linalg.norm(self.weights - prev_weights) < self.tol:
                break

    def predict(self, X):
        """
        Метод для предсказаний линейной регрессии
        X - матрица признаков
        """
        # Добавляем столбец единиц для учета свободного члена
        X = np.hstack((np.ones((X.shape[0], 1)), X))
        return np.dot(X, self.weights)


In [9]:
my_reg = LinearRegression(alpha=0.1, l_ratio=0.001, tol=0.001, max_iter=1000)
my_reg.fit(X, y)
mse_my_reg = mean_squared_error(y, my_reg.predict(X))
print(f"MSE: {mse_my_reg}")
assert mse_my_reg < 1e-3
print('You are amazing! Great work!')

MSE: 1.4001446771940016e-05
You are amazing! Great work!


***Задание 5 (1 балл)***. Обучите линейную регрессию из коробки

* с l1-регуляризацией (from sklearn.linear_model import Lasso)
* со значением параметра регуляризации 0.1

Обучите вашу линейную регрессию с тем же значением параметра регуляризации и сравните результаты. Сделайте выводы.

In [10]:
from sklearn.linear_model import Lasso
lasso_reg = Lasso(alpha=0.1).fit(X, y)
mse_lasso_reg = mean_squared_error(y, lasso_reg.predict(X))

my_reg = LinearRegression(alpha=0.1, l_ratio=0.1, tol=0.001, max_iter=1000)
my_reg.fit(X, y)
mse_my_reg = mean_squared_error(y, my_reg.predict(X))

print(f'MSE (Lasso) = {mse_lasso_reg};\nMSE (MyReg) = {mse_my_reg}')

MSE (Lasso) = 0.09926767786325631;
MSE (MyReg) = 0.10252518632022598


Библиотечная регрессия показала лучший ```MSE```. Возможных причин масса: более оптимальное значение ```learning_rate```, различия в способе инициализации весов, больше итераций обучения и т. д.